In [1]:
import pandas as pd
import openpyxl
print('openpyxl version:',openpyxl.__version__)
from sklearn.linear_model import LinearRegression

openpyxl version: 3.1.5


In [2]:
df = pd.read_excel(r'test_case_data_analyst.xlsx')

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114200 entries, 0 to 114199
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   product_id           114200 non-null  object        
 1   quantity             114200 non-null  int64         
 2   is_trial_period      114200 non-null  bool          
 3   purchase_date        114200 non-null  datetime64[ns]
 4   user_id              114200 non-null  int64         
 5   first_purchase_date  114200 non-null  datetime64[ns]
 6   cohort_month         114200 non-null  period[M]     
 7   days                 114200 non-null  int64         
 8   quantity_amount      114200 non-null  int64         
 9   revenue              114200 non-null  float64       
dtypes: bool(1), datetime64[ns](2), float64(1), int64(4), object(1), period[M](1)
memory usage: 8.0+ MB


In [4]:
df.isnull().sum()

product_id         0
quantity           0
is_trial_period    0
purchase_date      0
user_id            0
dtype: int64

In [5]:
df.head(10)

,product_id,quantity,is_trial_period,purchase_date,user_id
0,1week_4.99_USD_subscription_7days_trial,1,False,2020-02-19,9484222
1,1week_4.99_USD_subscription_7days_trial,1,False,2020-02-17,9422402
2,1week_4.99_USD_subscription_7days_trial,1,False,2020-02-18,4997991
3,1week_4.99_USD_subscription_7days_trial,1,False,2020-02-18,9455582
4,1week_4.99_USD_subscription_7days_trial,1,False,2020-02-16,8803657
5,1week_4.99_USD_subscription_7days_trial,1,False,2020-02-17,8800360
6,1week_4.99_USD_subscription_7days_trial,1,False,2020-02-17,8827207
7,1week_4.99_USD_subscription_7days_trial,1,False,2020-02-17,9202595
8,1week_4.99_USD_subscription_7days_trial,1,False,2020-02-17,9029259
9,1week_4.99_USD_subscription_7days_trial,1,False,2020-02-18,9456830


In [6]:
df[df['is_trial_period'] == True].count()

product_id         63008
quantity           63008
is_trial_period    63008
purchase_date      63008
user_id            63008
dtype: int64

In [7]:
df.user_id.nunique()

62909

In [24]:
first_order_df = df[df['is_trial_period'] == True].groupby('user_id').agg(first_purchase_date = ('purchase_date','min'))
first_order_df

,first_purchase_date
user_id,
404,2020-02-03
5337,2020-02-17
9010,2020-01-27
19880,2020-01-30
23403,2020-01-20
...,...
9694642,2020-02-20
9694662,2020-02-20
9694699,2020-02-20


In [9]:
first_order_df['cohort_month']=first_order_df['first_purchase_date'].dt.to_period('M')

In [10]:
df = df.merge(first_order_df, on = 'user_id')

In [11]:
df['days']=(df.purchase_date - df.first_purchase_date).dt.days
df

,product_id,quantity,is_trial_period,purchase_date,user_id,first_purchase_date,cohort_month,days
0,1week_4.99_USD_subscription_7days_trial,1,False,2020-02-19,9484222,2020-02-12,2020-02,7
1,1week_4.99_USD_subscription_7days_trial,1,False,2020-02-17,9422402,2020-02-10,2020-02,7
2,1week_4.99_USD_subscription_7days_trial,1,False,2020-02-18,4997991,2020-02-11,2020-02,7
3,1week_4.99_USD_subscription_7days_trial,1,False,2020-02-18,9455582,2020-02-11,2020-02,7
4,1week_4.99_USD_subscription_7days_trial,1,False,2020-02-16,8803657,2020-01-19,2020-01,28
...,...,...,...,...,...,...,...,...
114195,1week_4.99_USD_subscription_7days_trial,1,True,2020-01-04,8291100,2020-01-04,2020-01,0
114196,1week_4.99_USD_subscription_7days_trial,1,True,2020-01-02,8236936,2020-01-02,2020-01,0
114197,1week_4.99_USD_subscription_7days_trial,1,True,2020-01-03,8247910,2020-01-03,2020-01,0
114198,1week_4.99_USD_subscription_7days_trial,1,True,2020-01-04,8294146,2020-01-04,2020-01,0


In [12]:
quantity_amount = df.groupby('user_id').agg(quantity_amount=('quantity', 'sum')).reset_index()
df = df.merge(quantity_amount, on='user_id')
df

,product_id,quantity,is_trial_period,purchase_date,user_id,first_purchase_date,cohort_month,days,quantity_amount
0,1week_4.99_USD_subscription_7days_trial,1,False,2020-02-19,9484222,2020-02-12,2020-02,7,2
1,1week_4.99_USD_subscription_7days_trial,1,False,2020-02-17,9422402,2020-02-10,2020-02,7,2
2,1week_4.99_USD_subscription_7days_trial,1,False,2020-02-18,4997991,2020-02-11,2020-02,7,2
3,1week_4.99_USD_subscription_7days_trial,1,False,2020-02-18,9455582,2020-02-11,2020-02,7,2
4,1week_4.99_USD_subscription_7days_trial,1,False,2020-02-16,8803657,2020-01-19,2020-01,28,5
...,...,...,...,...,...,...,...,...,...
114195,1week_4.99_USD_subscription_7days_trial,1,True,2020-01-04,8291100,2020-01-04,2020-01,0,7
114196,1week_4.99_USD_subscription_7days_trial,1,True,2020-01-02,8236936,2020-01-02,2020-01,0,1
114197,1week_4.99_USD_subscription_7days_trial,1,True,2020-01-03,8247910,2020-01-03,2020-01,0,1
114198,1week_4.99_USD_subscription_7days_trial,1,True,2020-01-04,8294146,2020-01-04,2020-01,0,1


In [13]:
df['revenue'] = df['quantity_amount'] * 4.99
df

,product_id,quantity,is_trial_period,purchase_date,user_id,first_purchase_date,cohort_month,days,quantity_amount,revenue
0,1week_4.99_USD_subscription_7days_trial,1,False,2020-02-19,9484222,2020-02-12,2020-02,7,2,9.98
1,1week_4.99_USD_subscription_7days_trial,1,False,2020-02-17,9422402,2020-02-10,2020-02,7,2,9.98
2,1week_4.99_USD_subscription_7days_trial,1,False,2020-02-18,4997991,2020-02-11,2020-02,7,2,9.98
3,1week_4.99_USD_subscription_7days_trial,1,False,2020-02-18,9455582,2020-02-11,2020-02,7,2,9.98
4,1week_4.99_USD_subscription_7days_trial,1,False,2020-02-16,8803657,2020-01-19,2020-01,28,5,24.95
...,...,...,...,...,...,...,...,...,...,...
114195,1week_4.99_USD_subscription_7days_trial,1,True,2020-01-04,8291100,2020-01-04,2020-01,0,7,34.93
114196,1week_4.99_USD_subscription_7days_trial,1,True,2020-01-02,8236936,2020-01-02,2020-01,0,1,4.99
114197,1week_4.99_USD_subscription_7days_trial,1,True,2020-01-03,8247910,2020-01-03,2020-01,0,1,4.99
114198,1week_4.99_USD_subscription_7days_trial,1,True,2020-01-04,8294146,2020-01-04,2020-01,0,1,4.99


In [14]:
#Проверка правильности логики решения
df[df['user_id'] == 8291100].sort_values('purchase_date')

,product_id,quantity,is_trial_period,purchase_date,user_id,first_purchase_date,cohort_month,days,quantity_amount,revenue
114195,1week_4.99_USD_subscription_7days_trial,1,True,2020-01-04,8291100,2020-01-04,2020-01,0,7,34.93
71604,1week_4.99_USD_subscription_7days_trial,1,False,2020-01-11,8291100,2020-01-04,2020-01,7,7,34.93
21683,1week_4.99_USD_subscription_7days_trial,1,False,2020-01-18,8291100,2020-01-04,2020-01,14,7,34.93
86819,1week_4.99_USD_subscription_7days_trial,1,False,2020-01-25,8291100,2020-01-04,2020-01,21,7,34.93
4384,1week_4.99_USD_subscription_7days_trial,1,False,2020-02-01,8291100,2020-01-04,2020-01,28,7,34.93
81124,1week_4.99_USD_subscription_7days_trial,1,False,2020-02-08,8291100,2020-01-04,2020-01,35,7,34.93
52663,1week_4.99_USD_subscription_7days_trial,1,False,2020-02-15,8291100,2020-01-04,2020-01,42,7,34.93


In [15]:
cohort_revenue = df.groupby(['cohort_month', 'user_id']).agg(total_revenue=('revenue', 'max'))
final_cohort_revenue = cohort_revenue.groupby('cohort_month').agg(total_cohort_revenue=('total_revenue', 'sum')).reset_index()
final_cohort_revenue

,cohort_month,total_cohort_revenue
0,2020-01,415083.17
1,2020-02,154774.83


In [16]:
total_cohort_users = df.groupby('cohort_month')['user_id'].nunique().reset_index(name='total_users')
total_cohort_users

,cohort_month,total_users
0,2020-01,38494
1,2020-02,24415


In [17]:
total_cohort_users['total_users'].sum()

62909

In [18]:
#LTV = Общий доход / Количество клиентов

In [19]:
final_cohort_revenue = final_cohort_revenue.merge(total_cohort_users, on = 'cohort_month')

In [20]:
final_cohort_revenue.columns

Index(['cohort_month', 'total_cohort_revenue', 'total_users'], dtype='object')

In [21]:
final_cohort_revenue['LTV'] = final_cohort_revenue['total_cohort_revenue']/final_cohort_revenue['total_users']
final_cohort_revenue['LTV'] = final_cohort_revenue['LTV'].round(2)
final_cohort_revenue

,cohort_month,total_cohort_revenue,total_users,LTV
0,2020-01,415083.17,38494,10.78
1,2020-02,154774.83,24415,6.34


In [22]:
#Прогноз LTV на полгода
